In [6]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests

In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-07-02 00:07:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-02 00:07:28 (7.52 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [14]:
from groq import Groq

completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "user",
            "content": "give a c++ code for generating n th fibonacci number\n"
        },
    ],
        
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Here is a simple C++ function that generates the nth Fibonacci number using recursion:

```cpp
#include<iostream>
using namespace std;

unsigned long long fibonacci(int n) {
    if(n <= 1) {
        return n;
    } else {
        return fibonacci(n-1) + fibonacci(n-2);
    }
}

int main() {
    int n;
    cout<<"Enter the value of n: ";
    cin>>n;
    cout<<"The "<<n<<"th Fibonacci number is: "<<fibonacci(n)<<"\n";
    return 0;
}
```

Please note that this code uses recursion, which can be less efficient than other methods for larger values of `n` because it creates a lot of function calls on the stack. Also, the Fibonacci sequence grows very quickly, so even for moderately large values of `n`, the result may be too large to fit in an `unsigned long long` and may cause an overflow.

If you want a more efficient and safe solution for larger values of `n`, you can use an iterative approach to calculate the Fibonacci numbers:

```cpp
#include<iostream>
using namespace std;

unsigned lon

In [15]:
# Fetch the FAQ data
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

# Process the documents
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [16]:
documents[:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

In [18]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import NotFoundError

es_client = Elasticsearch('http://localhost:9200') 
print(es_client.info())

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
try:
    response = es_client.indices.delete(index=index_name)
    print(f"Deleted index: {index_name}")
except NotFoundError:
    print(f"Index {index_name} not found, nothing to delete")

es_client.indices.create(index=index_name, body=index_settings)

{'name': '61ab11b3e81e', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3sNPyls1R56dkPobwh87Eg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
Index course-questions not found, nothing to delete


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [19]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/mnt/E/projects/llm_zoomcamp/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:08<00:00, 116.30it/s]


In [20]:
def elastic_search(query, size=5, course=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    if course:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": course
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    # Step 4: Extract the _score field from the top-ranking result
    top_score = response['hits']['hits'][0]['_score']
    print("The top-ranking result _score is:", top_score)
    
    return result_docs

# Question-3

In [25]:
query = 'How do I execute a command in a running docker container?'

In [26]:
results = elastic_search(query, 5, None)
print(results)

The top-ranking result _score is: 84.050095
[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for 

# Question 4

In [27]:
course = 'machine-learning-zoomcamp'
size = 3

# Perform the search
results = elastic_search(query, size=size, course=course)
print(results)
# Check if we have at least 3 results and print the third question
if len(results) >= 3:
    third_question = results[2]['question']
    print("The third question returned by the search engine is:", third_question)
else:
    print("There are less than 3 results.")

The top-ranking result _score is: 84.050095
[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local

In [29]:
# check if we have at least 3 results and print all the questions with their rank

if results:
    for i, result in enumerate(results):
        print(f"Rank {i+1}: {result['question']}")
    if len(results) >= 3:
        third_question = results[2]['question']
        print("\nThe third question returned by the search engine is:", third_question)
    else:
        print("\nThere are less than 3 results.")
else:
    print("No results found.")


Rank 1: How do I debug a docker container?
Rank 2: How do I copy files from my local machine to docker container?
Rank 3: How do I copy files from a different folder into docker container’s working directory?

The third question returned by the search engine is: How do I copy files from a different folder into docker container’s working directory?


# Question 5

In [30]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt, context

In [31]:
def rag(query, size=3, course=None):
    search_results = elastic_search(query, size, course)
    prompt, context = build_prompt(query, search_results)
    return prompt, context

In [32]:
# Example usage to get the prompt length
query = 'How do I execute a command in a running docker container?'
course = 'machine-learning-zoomcamp'
size = 3

# Perform the search
results = elastic_search(query, size=size, course=course)

# Build the prompt using the search results
prompt, context = build_prompt(query, results)
print("Prompt is:", prompt)

# print(context)

# Calculate the length of the prompt
prompt_length = len(prompt)
print("The length of the resulting prompt is:", prompt_length)

The top-ranking result _score is: 84.050095
Prompt is: You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
d

In [33]:
!pip install tiktoken

  Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 700.0 kB/s eta 0:00:00m eta 0:00:010:00:01
Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (785 kB)


In [35]:
import tiktoken



In [36]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
print("The number of tokens in the resulting prompt is:", num_tokens)

The number of tokens in the resulting prompt is: 322
